In [18]:
import os 
import re
import sys
sys.path.append('..')
from classes.Parser import Parser
import pandas as pd

In [19]:
with open('../test_data/case1.py') as f:
    lines = f.readlines()
lines


['\n',
 'def func1(a,b):\n',
 '    c = a + b \n',
 '    return c\n',
 '\n',
 'a = 3\n',
 'a = 1\n',
 'd = 0\n',
 'b = 3 + d + a\n',
 'c = 10\n',
 'd = 14\n',
 't = a *3\n',
 'gaba = a+4\n',
 'for i in range(1):\n',
 '    for j in range( a + b ):\n',
 '        print(gaba)']

In [20]:
pars = Parser()
data =  pd.DataFrame(pars.get_data(lines)).T
data

,original_text,level_num,kind,line_text,name,arguments,result
0,\n,0,None,,None,[],NaN
1,"def func1(a,b):\n",0,def,"def func1(a,b):",func1,None,NaN
2,c = a + b \n,1,None,c = a + b,None,"[a, b]",c
3,return c\n,1,return,return c,None,[c],NaN
4,\n,0,None,,None,[],NaN
5,a = 3\n,0,None,a = 3,None,[3],a
6,a = 1\n,0,None,a = 1,None,[1],a
7,d = 0\n,0,None,d = 0,None,[0],d
8,b = 3 + d + a\n,0,None,b = 3 + d + a,None,"[3, d, a]",b
9,c = 10\n,0,None,c = 10,None,[10],c


In [27]:
n_data = parse_functions(data)
n_data

,original_text,level_num,kind,line_text,name,arguments,result,functions
0,\n,0,None,,None,[],NaN,main
1,"def func1(a,b):\n",0,def,"def func1(a,b):",func1,None,NaN,main
2,c = a + b \n,1,None,c = a + b,None,"[a, b]",c,func1
3,return c\n,1,return,return c,None,[c],NaN,func1
4,\n,0,None,,None,[],NaN,main
5,a = 3\n,0,None,a = 3,None,[3],a,main
6,a = 1\n,0,None,a = 1,None,[1],a,main
7,d = 0\n,0,None,d = 0,None,[0],d,main
8,b = 3 + d + a\n,0,None,b = 3 + d + a,None,"[3, d, a]",b,main
9,c = 10\n,0,None,c = 10,None,[10],c,main


In [39]:
def take_arguments_from_box(data,box):
    arg = []
    for i in range(box[0],box[1]):
        n_args = [ find_argument(data,x,i,data.iloc[i].functions) for x in data.iloc[i].arguments if not x.isnumeric() and "'" not in x and '"' not in x]
        arg = arg + n_args
    return arg

def take_argument_from_box(data,box):
    arg = []
    for i in range(box[0],box[1]):
        n_args = [ find_argument(data,x,i,data.iloc[i].functions) for x in data.iloc[i].arguments if not x.isnumeric() and "'" not in x and '"' not in x]
        arg = arg + n_args
    return arg
    
    
def find_argument(data,argument,line_num, function):
    _ = data.loc[(data.result == argument )& (data.functions == function)]
    potential_lines = [x for x in _.index.to_list() if x < line_num]
    potential_lines.sort()
    return potential_lines[-1]

take_arguments_from_box(n_data,(8,11))

[7, 6]

In [70]:
def take_prerequesits(data,line_num):
    
    def _help_func(data,line_num):
        arguments = data.iloc[line_num].arguments
        prereq = data.loc[data.result.isin(arguments)].index.to_list()
        prereq = [x for x in prereq if x < line_num]
        if len(prereq) == 0:
            return []
        grand = []
        for i in prereq:
                _ = _help_func(data,i)
                if len(_) != 0:
                    grand.append(_[-1])
                    
        rv =  grand + prereq
        return rv
    prereq = _help_func(data,line_num)
    temp_lines = data.iloc[prereq]
    rv = []
    for res in temp_lines.result.unique():
        result_type_list = temp_lines.loc[temp_lines.result == res].index.to_list()
        result_type_list.sort()
        rv.append(result_type_list[-1])
    
    return rv
    
data.iloc[pd.unique(take_prerequesits(data,14))]

,original_text,level_num,kind,line_text,name,arguments,result,functions
7,d = 0\n,0,None,d = 0,None,[0],d,main
6,a = 1\n,0,None,a = 1,None,[1],a,main
8,b = 3 + d + a\n,0,None,b = 3 + d + a,None,"[3, d, a]",b,main


In [41]:
l = [1]
l[-1]

1

In [22]:
def parse_functions(data):
    last_f_name = {}
    last = ['main']
    for i,row in data.iterrows():
        #print(row['name'],row['level_num'],last)
        
        if row.kind == 'def' or row.kind == 'class':
            last.append(row['name'])
            #print('tutaj',row['name'])
        if row['level_num'] == 0:
            last_f_name[i] = 'main'
        else:
            #print('tutaj')
            last_f_name[i] = last[-1]
            
        if row['kind'] == 'return':
            last.pop()
    data['functions'] = pd.Series(last_f_name)
    return data
            
parse_functions(data)          
        

,original_text,level_num,kind,line_text,name,arguments,result,functions
0,\n,0,None,,None,[],NaN,main
1,"def func1(a,b):\n",0,def,"def func1(a,b):",func1,None,NaN,main
2,c = a + b \n,1,None,c = a + b,None,"[a, b]",c,func1
3,return c\n,1,return,return c,None,[c],NaN,func1
4,\n,0,None,,None,[],NaN,main
5,a = 3\n,0,None,a = 3,None,[3],a,main
6,a = 1\n,0,None,a = 1,None,[1],a,main
7,d = 0\n,0,None,d = 0,None,[0],d,main
8,b = 3 + d + a\n,0,None,b = 3 + d + a,None,"[3, d, a]",b,main
9,c = 10\n,0,None,c = 10,None,[10],c,main


In [87]:
def take_prerequesits_from_box(data,box):
    list_of_lines = data.loc[data.functions == data.iloc[box[1]].functions].index.to_list()
    list_of_lines = [x for x in list_of_lines if x > box[1]]
    prereq = []
    for i in list_of_lines:
        f = data.iloc[i].functions
        for arg in data.iloc[i].arguments:
            if not arg.isnumeric() and "'" not in arg and '"' not in arg:
                prereq = prereq + [find_argument(data,arg,i,f)]
    
    prereq = pd.unique(prereq).tolist()
    prereq = [x for x in prereq if x in range(box[0],box[1])]
    return prereq

take_prerequesits_from_box(n_data,(7,9))

[8]

In [119]:
def make_function(data,start_line,end_line,name):
    if data.iloc[start_line].level_num != data.iloc[end_line + 1].level_num or data.iloc[start_line].functions != data.iloc[end_line + 1].functions:
        return False
    returns = take_prerequesits_from_box(data,(start_line,end_line))
    arguments = take_arguments_from_box(data,(start_line,end_line))
    arguments = data.iloc[arguments].result.to_list()
    returns = data.iloc[returns].result.to_list()
    if len(returns) == 0:
        pass
    replace = "'"
    
    start_index = data.loc[data.functions == data.iloc[start_line].functions].index.to_list()[0]
    def_string = f"def {name}({str(arguments)[1:-1].replace(replace,'')}):"
    start_level = data.iloc[start_line].level_num
    fun ={start_index: [def_string,start_level]} 
    
    start_index_backup = start_index 
    
    
    for i in range(start_line,end_line):
        start_index +=1
        fun[start_index] = [data.iloc[i].line_text,start_level +1]
    
    if len(returns) != 0:
        start_index +=1
        fun[start_index] = [f"return {str(returns)[1:-1].replace(replace,'')}",start_level+1]
        func_str = f"{str(returns)[1:-1].replace(replace,'')} = {name}({str(arguments)[1:-1].replace(replace,'')})"
    else:
        func_str = f"{name}({str(arguments)[1:-1].replace(replace,'')})"
        
    envoction = pd.DataFrame([{'line_text':func_str,'level_num':start_level}])
    
       
    
    
    my_function = pd.DataFrame(fun).T
    my_function.columns=['line_text','level_num']
    
    po = data.loc[data.index >  end_line][['line_text','level_num']]
    przed = data.loc[data.index <  start_line][['line_text','level_num']]
        
   
    #print(func_str)
    changed = pd.concat([przed,envoction,po]).reset_index(drop=True)
    przed = changed.iloc[changed.index < start_index_backup]
    po = changed.iloc[changed.index > start_index_backup]
    return pd.concat([przed,my_function,po]).reset_index(drop=True)
    


    #print(f"return {str(returns)[1:-1].replace(replace,'')}")
make_function(data,7,9,'lala')

,line_text,level_num
0,"def lala(d, a):",0
1,d = 0,1
2,b = 3 + d + a,1
3,return b,1
4,"def func1(a,b):",0
5,c = a + b,1
6,return c,1
7,,0
8,a = 3,0
9,a = 1,0


In [4]:
lista = [1,1,12,3]
lista.pop()
lista

[1, 1, 12]

In [ ]:
def get_functions

In [5]:
_ = data.iloc[:9]
data.iloc[:3]

,original_text,level_num,kind,line_text,name,arguments,result,functions
0,\n,0,None,,None,[],NaN,main
1,"def func1(a,b):\n",0,def,"def func1(a,b):",func1,None,NaN,main
2,c = a + b \n,1,None,c = a + b,None,"[a, b]",c,func1


In [104]:
def look_for_assign(line:str):
    if '=' in  line:
        splits = line.split('=')
        splits[0] = splits[0].replace(' ','')
        return True,splits
    else:
        return False, 0

In [106]:
def get_level(line):
    return line.count('    ')

In [118]:
def split_by_operators(line):
    operators = ['-','+','*','/']
    for operator in operators:
        line = line.replace(operator,'___TEMPORARY_SPLIT___')
    line = line.replace(' ','')
    return line.split('___TEMPORARY_SPLIT___')

In [138]:
def get_function(line):
    if line[0:5] == 'class':
        
        if '(' in line and ')' in line:
            parent = line[line.find("(")+1:line.find(")")]
            class_name = line[5:line.find("(")]
        else:
            parent = None
            class_name = line[5:]
        return 'class', class_name,parent
    elif line[0:3] == 'for':
        iterators = line[line.find("for")+3:line.find("in")]
        iterators = iterators.replace(" ",'')
        iterators = iterators.split(',')
        arguments = None
        if '(' and ')' in line:
            arguments = line[line.find("(")+1:line.find(")")]
            arguments = split_by_operators(arguments)
            if arguments[0] == '':
                arguments = None
        return 'for', iterators,arguments
    elif line[0:3] == 'def':
        name = line[line.find("def")+3:line.find("(")]
        return 'def',name,None
    elif '('in line and ')' in line:
        name =line[:line.find("(")]
        arguments = line[line.find("(")+1:line.find(")")]
        arguments = split_by_operators(arguments)
        if arguments[0] == '':
            arguments = None
        return 'func',name,arguments
    return None,None,None
    
get_function('a.print()')


('func', 'a.print', None)

In [139]:
def get_data(lines):
    data = {}
    for i,line in enumerate(lines):
        row = {}
        row['original_text'] = line
        row['level_num'] = get_level(line)
        row['kind'] = None
        line = line.replace('    ','')
        line = line.replace('\n','')
        row['line_text'] = line
        assign_flag,ass_res = look_for_assign(line)
        if assign_flag:
            row["resoult"] = ass_res[0]
            line = ass_res[1]
        kind,name,arguments = get_function(line)
        row['kind'] = kind
        row['name'] = name
        if kind is None:
            arguments = split_by_operators(line)
        row['arguments'] = arguments
        data[i] = row
        #print (split_by_operators(line), data['original_text'])
        #split_by_operators(line)
    return data
        
get_data(lines)

{0: {'original_text': '\n',
  'level_num': 0,
  'kind': None,
  'line_text': '',
  'name': None,
  'arguments': ['']},
 1: {'original_text': 'a = 1\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'a = 1',
  'resoult': 'a',
  'name': None,
  'arguments': ['1']},
 2: {'original_text': 'b = 4\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'b = 4',
  'resoult': 'b',
  'name': None,
  'arguments': ['4']},
 3: {'original_text': 'c = 10\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'c = 10',
  'resoult': 'c',
  'name': None,
  'arguments': ['10']},
 4: {'original_text': 'd = 14\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'd = 14',
  'resoult': 'd',
  'name': None,
  'arguments': ['14']},
 5: {'original_text': 't = a *3\n',
  'level_num': 0,
  'kind': None,
  'line_text': 't = a *3',
  'resoult': 't',
  'name': None,
  'arguments': ['a', '3']},
 6: {'original_text': 'gaba = a+4\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'gaba = a+4',
  'resoult': 'gaba',
  